# Burgers2D_DDBurgers2D
Demonstrate the domain decomposition formulation applied to the 2D Burgers equation.  
$\newcommand{\pdx}{\frac{\partial}{\partial x}}$
$\newcommand{\pdxx}{\frac{\partial^2}{\partial x^2}}$
$\newcommand{\pdy}{\frac{\partial}{\partial y}}$
$\newcommand{\pdyy}{\frac{\partial^2}{\partial y^2}}$
$\newcommand{\bb}{\boldsymbol{b}}$
$\newcommand{\br}{\boldsymbol{r}}$
$\newcommand{\bx}{\boldsymbol{x}}$
$\newcommand{\bw}{\boldsymbol{w}}$
$\newcommand{\bff}{\boldsymbol{f}}$
$\newcommand{\bu}{\boldsymbol{u}}$
$\newcommand{\bv}{\boldsymbol{v}}$
$\newcommand{\bB}{\boldsymbol{B}}$
$\newcommand{\bC}{\boldsymbol{C}}$
$\newcommand{\bI}{\boldsymbol{I}}$
$\newcommand{\bP}{\boldsymbol{P}}$
$\newcommand{\bmu}{\boldsymbol{\mu}}$
$\newcommand{\tbB}{\widetilde{\boldsymbol{B}}}$
$\newcommand{\tbC}{\widetilde{\boldsymbol{C}}}$
$\newcommand{\cD}{\mathcal{D}}$
$\newcommand{\real}{\mathbb{R}}$
$\newcommand{\bzero}{\boldsymbol{0}}$
$\newcommand{\set}[1]{\left\{#1\right\}}$

In [ ]:
import sys
with open("./../../PATHS.txt") as file:
  paths = file.read().splitlines()
sys.path.extend(paths)

In [ ]:
import os
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt

from time import time
from matplotlib import cm

In [ ]:
from dd_nm_rom.fom import Burgers2D, DDBurgers2D, Burgers2DExact

In [ ]:
plt.rc('font', size=20)
plt.rcParams['text.usetex'] = False
plt.rcParams['lines.markersize'] = 10

## 2D Burgers Equation
The 2D steady-state viscous Burgers Equation on $\Omega = [a, b] \times [c, d]\subset \real^2$ with viscosity $\nu>0$ and inhomogeneous Dirichlet boundary conditions is given by
\begin{align*}
    u \pdx u + v \pdy u &= \nu \left(\pdxx u + \pdyy u\right), \qquad (x, y)\in\Omega, \\
    u \pdx v + v \pdy v &= \nu \left(\pdxx v + \pdyy v\right), \qquad (x, y)\in\Omega,\\
    u(x, y) &= u_D(x, y), \qquad (x, y) \in \partial \Omega, \\
    v(x, y) &= v_D(x, y), \qquad (x, y) \in \partial \Omega. \\
\end{align*}
The domain is discretized uniformly with $n_x+2$ grid points in the $x$-direction and $n_y+2$ grid points in the $y$-direction, resulting in grid points $(x_i, y_j)$ where 
\begin{align*}
x_i = a + i h_x, \qquad i = 0, \dots, n_x+1, \\
y_j = c + j h_y, \qquad j = 0, \dots, n_y+1,
\end{align*}
where $h_x = (b-a)/(n_x+1)$ and $h_y = (d-c)/(n_y+1)$.
The solutions $u, v$ on the grid points are denoted $u_{ij} \approx u(x_i, y_j)$ and $v_{ij} \approx v(x_i, y_j)$. 
The PDE is then discretized using centered finite differences for the first and second derivative terms. The fully discretized system is given by
\begin{align*}
\bff_u(\bu, \bv) &= \bzero, \\
\bff_v(\bu, \bv) &= \bzero,
\end{align*}
where
\begin{align*}
\bu &=
\begin{bmatrix}
\bu^{[1]} \\ \vdots \\ \bu^{[n_y]}
\end{bmatrix} \in \real^{n_xn_y}, \qquad 
\bu^{[j]} = 
\begin{bmatrix}
u_{1, j} \\ \vdots \\ u_{n_x, j}
\end{bmatrix} \in \real^{n_x}, \quad j = 1, \dots, n_y,\\
\bv &=
\begin{bmatrix}
\bv^{[1]} \\ \vdots \\ \bv^{[n_y]}
\end{bmatrix} \in \real^{n_xn_y}, \qquad 
\bv^{[j]} = 
\begin{bmatrix}
v_{1, j} \\ \vdots \\ v_{n_x, j}
\end{bmatrix}\in \real^{n_x}, \quad j = 1, \dots, n_y,\\
 \bff_u(\bu, \bv)&= \bu \odot(\bB_x \bu - \bb_{ux1}) + \bv \odot(\bB_y \bu - \bb_{uy1}) \\
 &\quad + \bC_x \bu + \bb_{ux2} + \bC_y \bu + \bb_{uy2}, \\
  \bff_v(\bu, \bv)&= \bu \odot(\bB_x \bv - \bb_{vx1}) + \bv \odot(\bB_y \bv - \bb_{vy1}) \\
 &\quad + \bC_x \bv + \bb_{vx2} + \bC_y \bv + \bb_{vy2}, \\
\end{align*}
and where 
\begin{align*}
\bB_x &= -\frac{1}{2h_x} \left(\bI_{n_y} \otimes \tbB_x\right)\in \real^{n_xn_y \times n_x n_y}, \qquad
\tbB_x = 
\begin{bmatrix}
0 & 1 \\
-1 & \ddots & 1 \\
& -1 & 0
\end{bmatrix} \in \real^{n_x \times n_x}, \\
\bB_y &= -\frac{1}{2h_y}\left(\tbB_y \otimes \bI_{n_x}\right) \in \real^{n_xn_y \times n_x n_y} \qquad
\tbB_y = 
\begin{bmatrix}
0 & 1 \\
-1 & \ddots & 1 \\
& -1 & 0
\end{bmatrix} \in \real^{n_y \times n_y}, \\
\bC_x &= \frac{\nu}{h_x^2}\left(\bI_{n_y} \otimes \tbC_x\right) \in \real^{n_xn_y \times n_x n_y}, \qquad
\tbC_x = 
\begin{bmatrix}
-2 & 1 \\
1 & \ddots \\
& 1 & -2 \\
\end{bmatrix} \in \real^{n_x \times n_x}, \\
\bC_y &= \frac{\nu}{h_y^2} \left(\tbC_y \otimes \bI_{n_x}\right) \in \real^{n_xn_y \times n_x n_y}, \qquad
\tbC_y =
\begin{bmatrix}
-2 & 1 \\
1 & \ddots \\
& 1 & -2 \\
\end{bmatrix} \in \real^{n_y \times n_y}, \\
\bb_{ux1} &= -\frac{1}{2h_x} (\bb_{ux\ell} - \bb_{uxr}), \qquad
\bb_{uy1} = -\frac{1}{2h_y}(\bb_{uy\ell} - \bb_{uyr}), \\
\bb_{ux2} &= \frac{\nu}{h_x^2}(\bb_{ux\ell} + \bb_{uxr}), \qquad
\bb_{uy2} = \frac{\nu}{h_y^2} (\bb_{uy\ell} + \bb_{uyr}), \\
\bb_{vx1} &= -\frac{1}{2h_x} (\bb_{vx\ell} - \bb_{vxr}), \qquad
\bb_{vy1} = -\frac{1}{2h_y}(\bb_{vy\ell} - \bb_{vyr}), \\
\bb_{vx2} &= \frac{\nu}{h_x^2}(\bb_{vx\ell} + \bb_{vxr}), \qquad
\bb_{vy2} = \frac{\nu}{h_y^2} (\bb_{vy\ell} + \bb_{vyr}), \\
\bb_{ux\ell} &= 
\begin{bmatrix}
u_D(x_0, y_1) \\ \vdots \\ u_D(x_0, y_{n_y})
\end{bmatrix} \otimes 
\begin{bmatrix}
1 \\ 0 \\ \vdots \\ 0
\end{bmatrix}_{n_x \times 1} \in \real^{n_x n_y}, \qquad
\bb_{uxr} = 
\begin{bmatrix}
u_D(x_{n_x+1}, y_1) \\ \vdots \\ u_D(x_{n_x+1}, y_{n_y})
\end{bmatrix} \otimes 
\begin{bmatrix}
0 \\ \vdots \\ 0 \\ 1
\end{bmatrix}_{n_x \times 1} \in \real^{n_x n_y}, \\
\bb_{uyb} &= 
\begin{bmatrix}
1 \\ 0 \\ \vdots \\ 0
\end{bmatrix}_{n_y \times 1} 
\otimes 
\begin{bmatrix}
u_D(x_1, y_0) \\ \vdots \\ u_D(x_{n_x}, y_0)
\end{bmatrix} \in \real^{n_x n_y} \qquad
\bb_{uyt} = 
\begin{bmatrix}
0 \\ \vdots \\ 0 \\ 1
\end{bmatrix}_{n_y \times 1} 
\otimes 
\begin{bmatrix}
u_D(x_1, y_{n_y+1}) \\ \vdots \\ u_D(x_{n_x}, y_{n_y+1})
\end{bmatrix} \in \real^{n_x n_y} \\
\bb_{vx\ell} &= 
\begin{bmatrix}
v_D(x_0, y_1) \\ \vdots \\ v_D(x_0, y_{n_y})
\end{bmatrix} \otimes 
\begin{bmatrix}
1 \\ 0 \\ \vdots \\ 0
\end{bmatrix}_{n_x \times 1} \in \real^{n_x n_y}, \qquad
\bb_{vxr} = 
\begin{bmatrix}
v_D(x_{n_x+1}, y_1) \\ \vdots \\ v_D(x_{n_x+1}, y_{n_y})
\end{bmatrix} \otimes 
\begin{bmatrix}
0 \\ \vdots \\ 0 \\ 1
\end{bmatrix}_{n_x \times 1} \in \real^{n_x n_y}, \\
\bb_{vyb} &= 
\begin{bmatrix}
1 \\ 0 \\ \vdots \\ 0
\end{bmatrix}_{n_y \times 1} 
\otimes 
\begin{bmatrix}
v_D(x_1, y_0) \\ \vdots \\ v_D(x_{n_x}, y_0)
\end{bmatrix} \in \real^{n_x n_y} \qquad
\bb_{vyt} = 
\begin{bmatrix}
0 \\ \vdots \\ 0 \\ 1
\end{bmatrix}_{n_y \times 1} 
\otimes 
\begin{bmatrix}
v_D(x_1, y_{n_y+1}) \\ \vdots \\ v_D(x_{n_x}, y_{n_y+1})
\end{bmatrix} \in \real^{n_x n_y} 
\end{align*}

In [ ]:
# define constant parameters for PDE
nx, ny = 480, 24
x_lim  = [-1.0, 1.0]
y_lim  = [0.0, 0.05]
a1_lim    = [1.0, 10000.0]
lam_lim   = [5.0, 25.0]
a1 = 1e4
lam = 5.0
viscosity = 1e-1

fig_dir = f'/Users/zanardi1/Workspace/Codes/DD-NM-ROM/run/figures/nx_{nx}_ny_{ny}_mu_{viscosity}/'
os.makedirs(fig_dir, exist_ok=True)
    
# compute FOM for given a1 and lambda
burg_ex = Burgers2DExact()
burg_ex.set_params(a1, lam, viscosity)

# generate Burgers FOM
fom = Burgers2D(nx, ny, x_lim, y_lim, viscosity)
fom.set_bc(burg_ex.u, burg_ex.v)
x0 = np.zeros(fom.get_ndof())
uv, rhs = fom.solve(x0, tol=1e-9, maxit=20, stepsize_min=1e-20, verbose=True)
print("RUNTIME:", fom.runtime)

In [ ]:
# compute exact u and v on grid
x        = np.linspace(x_lim[0], x_lim[1], nx+2)[1:-1]
y        = np.linspace(y_lim[0], y_lim[1], ny+2)[1:-1]
X, Y     = np.meshgrid(x, y)
Uex, Vex = burg_ex.u(X, Y), burg_ex.v(X, Y)
# plot FD u and v
U = uv["u"].reshape(ny, nx)
V = uv["v"].reshape(ny, nx)

In [ ]:
# plot exact u and v
plt.figure(figsize=(12, 4))
plt.pcolormesh(X, Y, Uex, cmap=cm.jet, shading='auto')
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.title(f'$a_1 = {a1}, \lambda = {lam}$')
cb = plt.colorbar(label='$u_{ex}(x, y)$')
plt.show()

plt.figure(figsize=(12, 4))
plt.pcolormesh(X, Y, Vex, cmap=cm.jet, shading='auto')
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.title(f'$a_1 = {a1}, \lambda = {lam}$')
cb = plt.colorbar(label='$v_{ex}(x, y)$')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.pcolormesh(X, Y, U, cmap=cm.jet, shading='auto')
# plt.xlabel('$x$')
# plt.ylabel('$y$')
cb = plt.colorbar()#label='$u(x, y)$')
# plt.title(f'$a_1 = {a1}, \lambda = {lam}$')
file=fig_dir+f'u_fom_a1_{a1}_lam_{lam}.png'
plt.savefig(file, bbox_inches='tight', pad_inches=0.1)
plt.show()

plt.figure(figsize=(10, 6))
plt.pcolormesh(X, Y, V, cmap=cm.jet, shading='auto')
# plt.xlabel('$x$')
# plt.ylabel('$y$')
cb = plt.colorbar()#label='$v(x, y)$')
# plt.title(f'$a_1 = {a1}, \lambda = {lam}$')
file=fig_dir+f'v_fom_a1_{a1}_lam_{lam}.png'
plt.savefig(file, bbox_inches='tight', pad_inches=0.1)
plt.show()

In [ ]:
# compute relative error between exact and FD solutions
u_rel_err = np.linalg.norm(U-Uex)/np.linalg.norm(Uex)
v_rel_err = np.linalg.norm(V-Vex)/np.linalg.norm(Vex)
print(f'u relative error = {u_rel_err:1.4e}')
print(f'v relative error = {v_rel_err:1.4e}')

In [ ]:
# compute relative error between exact and FD solutions
u_rel_err = 100*np.mean(np.abs(U-Uex)/(np.abs(Uex)+1e-7))
v_rel_err = 100*np.mean(np.abs(V-Vex)/(np.abs(Vex)+1e-7))
print(f'u relative error = {u_rel_err:1.4e}')
print(f'v relative error = {v_rel_err:1.4e}')

## Domain-decomposition
The full-order model (FOM) can expressed as a parametrized system of nonlinear algebraic equations
\begin{equation}\label{eq:fom_residual}
    \br(\bx; \bmu) = \bzero,
\end{equation}
where $\br: \real^{N_x}\times \cD \to \real^{N_x}$ denotes the residual and is nonlinear in (at least) its first argument, $\bmu \in \cD \subset \real^{N_\mu}$
denotes the FOM parameters, and $\bx:\cD \to \real^{N_x}$ denotes the state.
For notational simplicity, the dependence on $\bmu$ is suppressed until needed.
Next consider a decomposition of the system of the FOM into $n_\Omega \leq N_x$ algebraic subdomains such that the residual satisfies
\begin{equation}\label{eq:fom_dd_residual}
  \br(\bw) = \sum_{i=1}^{n_\Omega} \left(\bP_i^r\right)^T \br_i(\bP_i^\Omega \bw, \bP_i^\Gamma \bw), \qquad \forall \; \bw  \in \real^{N_x},
\end{equation}
where $\br_i: \real^{N_i^\Omega} \times \real^{N_i^\Gamma} \to \real^{N_i^r}$ denotes the residual on the $i$th subdomain,
$\bP_i^r \in \set{0, 1}^{N_i^r \times N_x}$ denotes the $i$th residual sampling matrix,
$\bP_i^\Omega \in \set{0, 1}^{N_i^\Omega \times N_x}$ denotes the $i$th interior-state sampling matrix, and
$\bP_i^\Gamma \in \set{0, 1}^{N_i^\Gamma \times N_x}$ denotes the $i$th interface-state sampling matrix.
The variables 
$$
\bw_i^\Omega := \bP_i^\Omega \bw, \qquad
\bw_i^\Gamma := \bP_i^\Gamma \bw,
$$
are also referred to as the interior and interface states on the $i$th subdomain, respectively. 

Next the domain-decomposition formulation is illustrated on a coarse domain. The subdomain grids are visualized for each subdomain, including each of the interior and interface states and ports.

In [ ]:
# define constant parameters for PDE
nx, ny = 8, 8
num_sub_x = 2
num_sub_y = 2

# generate Burgers FOM on coarse grid for visualization
fom = Burgers2D(nx, ny, x_lim, y_lim, viscosity)
fom.set_bc(burg_ex.u, burg_ex.v)

# compute DD model
ddmdl = DDBurgers2D(fom, num_sub_x, num_sub_y)
ddmdl.set_bc()

In [ ]:
x = np.linspace(x_lim[0], x_lim[1], nx+2)[1:-1]
y = np.linspace(y_lim[0], y_lim[1], ny+2)[1:-1]
X, Y = np.meshgrid(x, y)
xx, yy = X.flatten(), Y.flatten()

# plot interior states
plt.figure(figsize=(10,10))
m = ['o', '^', 'd', 'P', '>', '*']
size = 300
for i, s in enumerate(ddmdl.subdomains):
    plt.scatter(xx[s.indices["interior"]], yy[s.indices["interior"]], marker=m[i%5], s=size, label=f'$\Omega_{i}$')
plt.tick_params(left = False, right = False , labelleft = False ,
                labelbottom = False, bottom = False)
plt.legend()
plt.title('Interior States')
plt.show()

In [ ]:
# plot interface states
plt.figure(figsize=(10,10))
for i, s in enumerate(ddmdl.subdomains):
    plt.scatter(xx[s.indices["interface"]], yy[s.indices["interface"]], s=size, marker=m[i%5], label=f'$\Omega_{i}$')
plt.tick_params(left = False, right = False , labelleft = False ,
                labelbottom = False, bottom = False)
plt.legend()
plt.title('Interface States')
plt.show()

In [ ]:
# plot ports
plt.figure(figsize=(10,10))
for i, port in ddmdl.port_to_nodes.items():
    plt.scatter(xx[port], yy[port], s=size, marker=m[i%5], label=f'{i}') #label=f'{list(port)}')
plt.tick_params(left = False, right = False , labelleft = False ,
                labelbottom = False, bottom = False)
# plt.legend(ncol=2)
file = fig_dir + '/dd_ports.png'
plt.savefig(file, bbox_inches='tight', pad_inches=0.1)
plt.show()

In [ ]:
# plot residual indices
plt.figure(figsize=(10,10))
for i, s in enumerate(ddmdl.subdomains):
    plt.scatter(xx[s.indices["res"]], yy[s.indices["res"]], s=size, marker='o', label=f'$\Omega_{i}$')
plt.tick_params(left = False, right = False , labelleft = False ,
                labelbottom = False, bottom = False)
file = fig_dir + '/dd_residuals.png'
plt.savefig(file, bbox_inches='tight', pad_inches=0.1)
plt.show()

In [ ]:
# plot subdomain indices
plt.figure(figsize=(10,10))
colors = ['blue', 'orange', 'green', 'red']
for i, s in enumerate(ddmdl.subdomains):
#     plt.scatter(xx[s.indices_interior], yy[s.indices_interior], s=size, color=colors[i%5], marker='o')
#     plt.scatter(xx[s.indices_interface], yy[s.indices_interface], s=size, color=colors[i%5], marker='P')
    
    xx_i = np.concatenate((xx[s.indices["interior"]], xx[s.indices["interface"]]))
    yy_i = np.concatenate((yy[s.indices["interior"]], yy[s.indices["interface"]]))
    plt.scatter(xx_i, yy_i, s=size, marker=m[i%5], label=f'$\Omega_{i}$')

plt.tick_params(left = False, right = False , labelleft = False ,
                labelbottom = False, bottom = False)
# plt.legend()
# plt.title('Subdomain states')
file = fig_dir + '/dd_states.png'
plt.savefig(file, bbox_inches='tight', pad_inches=0.1)
plt.show()

In [ ]:
# checks that constraint matrices are computed correctly
c = np.zeros(ddmdl.n_constraints)
vec = np.random.rand(2*fom.nxy)
for s in ddmdl.subdomains:
    c += s.cmat["interface"]@np.concatenate([vec[s.indices["interface"]],vec[fom.nxy+s.indices["interface"]]])
print('||sum(A[i] x[i])||=', np.linalg.norm(c))

# Compute state solutions u and v using DD model

In [ ]:
# define constant parameters for PDE
nx, ny = 120, 12
num_sub_x = 2
num_sub_y = 1

# generate Burgers FOM on coarse grid for visualization
fom = Burgers2D(nx, ny, x_lim, y_lim, viscosity)
fom.set_bc(burg_ex.u, burg_ex.v)
x0 = np.zeros(fom.get_ndof())
uv, res = fom.solve(x0, tol=1e-9, maxit=20, stepsize_min=1e-20, verbose=True)
print("RUNTIME:", fom.runtime)

In [ ]:
# compute DD model
ddmdl_s = DDBurgers2D(fom, num_sub_x, num_sub_y, constraint_type='strong')
ddmdl_s.set_bc()

# # Solve DD model
# ndof_s = 2*np.sum([s.n_nodes["interior"]+s.n_nodes["interface"] for s in ddmdl_s.subdomains]) \
#             + ddmdl_s.subdomains[0].cmat_intf.shape[0]
# w0_s = np.zeros(ndof_s)
w0_s = np.zeros(ddmdl_s.get_ndof())

uv_s, lambdas = ddmdl_s.solve(w0_s, tol=1e-6, maxit=50, 
  stepsize_min=1e-20, verbose=True)
print("RUNTIME:", ddmdl_s.runtime)


# uv_s, v_s, uv_s_intr, v_s_intr, uv_s_intf, v_s_intf, lam_s, strong_time, itr = ddmdl_s.solve(w0_s, tol=1e-5, 
#                                                                               maxit=15,
#                                                                               print_hist=True)

In [ ]:
# compute DD model
ddmdl_w = DDBurgers2D(fom, num_sub_x, num_sub_y, constraint_type='weak', n_constraints_weak=47, seed=0)
ddmdl_w.set_bc()

# # Solve DD model
# ndof_s = 2*np.sum([s.n_nodes["interior"]+s.n_nodes["interface"] for s in ddmdl_s.subdomains]) \
#             + ddmdl_s.subdomains[0].cmat_intf.shape[0]
# w0_s = np.zeros(ndof_s)
w0_w = np.zeros(ddmdl_w.get_ndof())

uv_w, lambdas_w = ddmdl_w.solve(w0_w, tol=1e-6, maxit=500, 
  stepsize_min=1e-20, verbose=True)
print("RUNTIME:", ddmdl_w.runtime)

In [ ]:
error = 0.0
for j in range(ddmdl_s.subs_indices.n_subs):
    num = np.sum(np.square(uv_s["interior"]["u"][j]-uv_w["interior"]["u"][j])) +\
          np.sum(np.square(uv_s["interior"]["v"][j]-uv_w["interior"]["v"][j])) +\
          np.sum(np.square(uv_s["interface"]["u"][j]-uv_w["interface"]["u"][j])) +\
          np.sum(np.square(uv_s["interface"]["v"][j]-uv_w["interface"]["v"][j]))
    den = np.sum(np.square(uv_s["interior"]["u"][j])) +\
          np.sum(np.square(uv_s["interior"]["v"][j])) +\
          np.sum(np.square(uv_s["interface"]["u"][j])) +\
          np.sum(np.square(uv_s["interface"]["v"][j]))
    error += num/den
error = np.sqrt(error/ddmdl_s.subs_indices.n_subs)
print(f'Error between solutions to strongly and weakly constrained FOMs = {error:1.4e}')

In [ ]:
# plot FD u and v
x = np.linspace(x_lim[0], x_lim[1], nx+2)[1:-1]
y = np.linspace(y_lim[0], y_lim[1], ny+2)[1:-1]
X, Y = np.meshgrid(x, y)
U_full = uv_w["res"]["u"].reshape(ny, nx)
V_full = uv_w["res"]["v"].reshape(ny, nx)

plt.figure(figsize=(12, 4))
plt.pcolormesh(X, Y, U_full, cmap=cm.jet, shading='auto')
plt.xlabel('$x$')
plt.ylabel('$y$')
cb = plt.colorbar(label='$u_{strong}(x, y)$')

plt.figure(figsize=(12, 4))
plt.pcolormesh(X, Y, V_full, cmap=cm.jet, shading='auto')
plt.xlabel('$x$')
plt.ylabel('$y$')
cb = plt.colorbar(label='$v_{strong}(x, y)$')

In [ ]:
# plot FD u and v
x = np.linspace(x_lim[0], x_lim[1], nx+2)[1:-1]
y = np.linspace(y_lim[0], y_lim[1], ny+2)[1:-1]
X, Y = np.meshgrid(x, y)
U_full = uv_s["res"]["u"].reshape(ny, nx)
V_full = uv_s["res"]["v"].reshape(ny, nx)

plt.figure(figsize=(12, 4))
plt.pcolormesh(X, Y, U_full, cmap=cm.jet, shading='auto')
plt.xlabel('$x$')
plt.ylabel('$y$')
cb = plt.colorbar(label='$u_{weak}(x, y)$')

plt.figure(figsize=(12, 4))
plt.pcolormesh(X, Y, V_full, cmap=cm.jet, shading='auto')
plt.xlabel('$x$')
plt.ylabel('$y$')
cb = plt.colorbar(label='$v_{weak}(x, y)$')

In [ ]:
# plot sparsity of jacobian
r, jac = ddmdl.rhs_jac(np.random.rand(ddmdl.get_ndof()))[:2]
plt.figure(figsize=(10, 10))
plt.spy(jac)
plt.show()

In [ ]:
# plot sparsity of constraint matrix
plt.figure(figsize=(10, 10))
plt.spy(ddmdl.subdomains[0].cmat["interface"])
plt.show()

In [ ]:
dd_u_rel_err = np.linalg.norm(uv_s["res"]["u"]-uv["u"])/np.linalg.norm(uv["u"])
dd_v_rel_err = np.linalg.norm(uv_s["res"]["v"]-uv["v"])/np.linalg.norm(uv["v"])
print('DD u relative error = ', dd_u_rel_err)
print('DD v relative error = ', dd_v_rel_err)